# PYTHON 3
## Lesson 4
### Пространства имен. Декораторы. PEP8. Jupyter Notebook.


# Пространства имен (Namespaces)

Это отображение из множества названий переменных в множество объектов

Циклы и условия не создают своё пространство имён

In [2]:
for i in range(3):
    a = i ** 2

print(a)

4


In [3]:
if True:
    a = 2
    
print(a)

2


locals() - возвращает текущий namespace в виде словаря

globals() - возвращает namespace модуля



In [4]:
def test():
    x = 1
    print(locals(), 'x' in globals(), 'x' in locals())

test()

{'x': 1} False True


Функции создают свой namespace

In [5]:
def f():
    in_func = 2

f()
in_func

NameError: ignored

### Область видимости (scope)

Правило LEGB:

1) Local - имена, определенные внутри функции (и не помеченные global)

2) Enclosing-function locals - имена в области видимости всех оборачивающих (enclosing) функций, в порядке уменьшения глубины

3) Global - глобальные имена, определенные на уровне модуля или посредством global

4) Built-in - предопределенные (range, open, ...)

Видимость доступа

In [50]:
x = 5

def f():
    print(x)

f()

5


Фейл при попытке изменения

In [2]:
x = 5

def f():
    x = 4

f()
print(x)

5


Ключевое слово **global**

In [3]:
x = 5

def f():
    global x
    x = 4

f()
print(x)

4


Конфликт с локальным аргументом

In [5]:
x = 5

def f(x):
    global x
    x = 4
  
f(x)

SyntaxError: ignored

Фейл для вложенных пространств имён

In [1]:
def f():
    x = 5
    def g():
        x = 4  # global ссылается на "глобальный" x, а не x из f
    g()
    print(x)
f()

5


Ключевое слово **nonlocal**

In [3]:
def f():
    x = 5
    def g():
        nonlocal x
        x = 4
    g()
    print(x)
f()

4


Замыкания как способ сохранения состояния

In [6]:
def make_adder(x):
    def adder(y):
        return x + y
    return adder

add5 = make_adder(5)
add5(5)

10

In [2]:
def cell(value = 0):
    def get():
        return value
    
    def set(new_value):
        nonlocal value
        value = new_value
        return value
    
    return get, set

get, set = cell(10)
print(get())

set(20)

print(get())

10
20


# Декораторы

In [9]:
def my_decorator(function_to_decorate):
    def wrapper():
        print("( ͡° ͜ʖ ͡°)")
        result = function_to_decorate()
        print("( ͡ᵔ ͜ʖ ͡ᵔ)")
        return result
    return wrapper

def f():
    print("╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ")

my_decorator(f)()

( ͡° ͜ʖ ͡°)
╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
( ͡ᵔ ͜ʖ ͡ᵔ)


Синтаксический сахар

In [10]:
@my_decorator
def f():
    print("ヽ( ⌒o⌒)人(⌒-⌒ )ﾉ")

f()

( ͡° ͜ʖ ͡°)
ヽ( ⌒o⌒)人(⌒-⌒ )ﾉ
( ͡ᵔ ͜ʖ ͡ᵔ)


Порядок вызовов

In [11]:
def my_decorator1(f):
    def wrapper():
        print("┬┴┬┴┤( ͡° ͜ʖ├┬┴┬┴")
        result = f()
        print("( ͡° ͜ʖ ͡°)")
        return result
    return wrapper

def my_decorator2(f):
    def wrapper():
        print("┬┴┬┴┤(･├┬┴┬┴")
        result = f()
        print("(･ω･)")
        return result
    return wrapper

@my_decorator1
@my_decorator2
def f():
    print("(づ ◕‿◕ )づ")
  
f()

┬┴┬┴┤( ͡° ͜ʖ├┬┴┬┴
┬┴┬┴┤(･├┬┴┬┴
(づ ◕‿◕ )づ
(･ω･)
( ͡° ͜ʖ ͡°)


Аргументы функции

In [12]:
def decorator(f):
    def wrapper(arg):
        print("( ͠° ͟ʖ ͡°)")
        f(arg)
        print("( ͡ᵔ ͜ʖ ͡ᵔ)")
    return wrapper

@decorator
def f(arg):
    print("(⊃｡•́‿•̀｡)⊃" +arg)

f("━✿✿✿✿✿✿")

( ͠° ͟ʖ ͡°)
(⊃｡•́‿•̀｡)⊃━✿✿✿✿✿✿
( ͡ᵔ ͜ʖ ͡ᵔ)


Передача всех аргументов

In [13]:
def perfect_decorator(f):
    def wrapper(*args, **kwargs):
        print("-------------------------")
        f(*args, **kwargs)
        print("-------------------------")
    return wrapper

@perfect_decorator
def f(man1, man2, *, table1, table2):
    print(man1, table1, man2, table2)
  
f("╮°-°)╮", "( ╯°□°)╯", table1="┳━━┳", table2="┻━━┻")

-------------------------
╮°-°)╮ ┳━━┳ ( ╯°□°)╯ ┻━━┻
-------------------------


Аргументы декоратора

In [14]:
def decorator_maker(arg):
    def param_decorator(f):
        def wrapper():
            return  f() + arg
        return wrapper
    return param_decorator
@decorator_maker("【　TV　】")
def f():
    return "(　￣.)o-"
print(f())

(　￣.)o-【　TV　】


Зачем оно бывает нужно

In [15]:
import time

def benchmark(func):
    def wrapper(*args, **kwargs):
        t = time.clock()
        res = func(*args, **kwargs)
        print(func.__name__, time.clock() - t)
        return res
    return wrapper

@benchmark
def f1():
    l = [i for i in range(1000000)]

@benchmark
def f2():
    l = []
    for i in range(1000000):
        l.append(i)

f1()
f2()

f1 0.08944299999999972
f2 0.1255090000000001


Проблема

In [9]:
def show(x):
    "This is a really nice looking docstring"
    print(x)

print(show.__name__)
print(show.__doc__)

show
This is a really nice looking docstring


In [10]:
def deprecated(func):
    def wrapper(*args, **kwargs):
        print('{} is deprecated!'.format(func.__name__), file=sys.stderr)
        return func(*args, **kwargs)
    return wrapper

@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)
    
print(show.__name__)
print(show.__doc__) 

wrapper
None


Решение - **functools.wraps**

In [11]:
import functools

def deprecated(func):
    @functools.wraps(func) 
    def wrapper(*args, **kwargs):
        print('{} is deprecated!'.format(func.__name__), file=sys.stderr)
        return func(*args, **kwargs)
    return wrapper

@deprecated
def show(x):
    'This is a really nice looking docstring'
    print(x)

print(show.__name__)
print(show.__doc__)

show
This is a really nice looking docstring


# PEP8



Методы класса - одной пустой строкой
Не нужно лишних пробелов
Имена:

*   Максимум 79 символов в строке
*   Глобальные функции и определения классов окружаются 2 пустыми строками
*   Методы класса - одной пустой строкой
*   Имена:
   *  Модули - short, all-lowercase, нижнее подчёркивание допустимо
   *  Классы - CapWords, типа MyClass
   *  Функции - all-lowercase, с нижними подчёркиваниями в качестве разделителя слов
   *  Локальные переменные, глобальные переменные - аналогично функциям
   *  Константы - все большие буквы, с нижними подчёркиваниями в качестве разделителя слов
* + много мелких парвил

Pylint (https://www.pylint.org/) или pep8 (https://pypi.python.org/pypi/pep8) для проверки

# Jupyter Notebook
Самый распростренный инструмент для анализа данных

Как запустить:

`jupyter notebook`

Основная единица работы - **.ipynb**-файл (в простонародии - "ноутбук")